In [42]:
import os
 #-----------------------------  open import ---------------------------------------#------ 
import cv2
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from skimage.io import imread,imshow
from skimage import measure,color,io,feature
from scipy.ndimage.morphology import binary_fill_holes
import matplotlib.patches as mpatches
from skimage.morphology import convex_hull_image
from skimage.measure import label, regionprops
import math
from scipy import ndimage as ndi
from skimage import feature
from skimage.filters import threshold_mean
from copy import copy,deepcopy
%matplotlib inline

In [43]:
x=[]
y=[]
feature=[]
path='C:/Users/Bermuda/OneDrive/Workshop/Dataset'
for fname in os.listdir(path):
#     print(fname,end=' ')
    paths=path+'/'+str(fname)
    for iname in os.listdir(paths):
        #print(iname,end=' ')
        pathss=paths+'/'+str(iname)
        img=imread(pathss)

        #convert rgb to hsv
        img_hsv=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)

        #select green only
        img_green=deepcopy(img_hsv)

        img_green[:,:,0]=0
        img_green[:,:,2]=0

        #convert hsv green only to grayscale
        img_gray0=img_green[:,:,1]

        #thresholding
        ret,thresh = cv2.threshold(img_gray0,50,255,cv2.THRESH_BINARY)

        #morprology (dilation)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3)) 
        dilation = cv2.dilate(thresh,kernel,iterations = 1)
        dilation[99:,:]=1
        dilation[30:60,99:]=1

        #fill holes
        filled_I = binary_fill_holes(dilation)
        filled_I[99:,:]=0  
        filled_I[30:60,99:]=0
        filled_I = filled_I*1
        
        im2,contours,h= cv2.findContours(filled_I,2,1)
        cnt = contours[0]
        hull = cv2.convexHull(cnt,returnPoints = False)
        hulla = cv2.convexHull(cnt,returnPoints = True)
        hullarea = cv2.contourArea(hulla)
        conarea = cv2.contourArea(cnt)
        defects = cv2.convexityDefects(cnt,hull)
        
        #find center
        M = cv2.moments(cnt)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        center = (cx,cy+10)
        cv2.circle(img,center,1,[255,0,0],-1)

        count_defects = 0
        count_finger = 0
        lena = []
        lenfinger = []
        anglelist = []

        maxi = 0
        check = 0
        prelen=0
        
        count_defects = 0
        count_finger = 0
        lena = [0,0,0,0]
        lenfinger = []
        anglelist = [0,0,0,0]
            
        #find defect
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])

            # find length of all sides of triangle
            #start to end
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            #start to defect
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            #defect to end
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

            #center to start check 1 
            longlen = math.sqrt((start[0] - cx)**2 + (start[1] - cy)**2)
            if longlen > maxi:
                maxi = longlen
                tip = start
        
            
            # ignore angles > 90 and highlight rest with red dots
            if angle <= 92:
                anglelist.append(angle)
                count_defects += 1
                cv2.circle(img, far, 1, [255,0,127], -1)
                #dist = cv2.pointPolygonTest(cnt,far,True)

                # draw a line from start to end i.e. the convex points (finger tips)
                # (can skip this part)

                cv2.line(img,start, end, [255,153,153], 2)
                cv2.circle(img, start, 1, [153,51,255], -1)
                cv2.circle(img, end, 1, [153,51,255], -1)
                lena.append(a)
                longlen1 = math.sqrt((start[0] - cx)**2 + (start[1] - cy)**2)
                longlen2 = math.sqrt((end[0] - cx)**2 + (end[1] - cy)**2)
                if count_defects == 1:
                    lenfinger.append(longlen1)
                    lenfinger.append(longlen2)
                    cv2.line(img,center,start,[255,0,0],1)
                    cv2.line(img,center,end,[255,0,0],1)
                    count_finger+=2
                else:
                    lenfinger.append(longlen2)
                    cv2.line(img,center,end,[255,0,0],1)
                    count_finger+=1
        if count_defects == 0:
            if tip[1]<cy:
                cv2.circle(img, tip, 1, [153,51,255], -1)
                cv2.line(img,center,tip,[255,0,0],1)
                count_finger+=1
                
#         print(count_finger)
        # print(count_defects)
        # print(conarea)
        # print(hullarea)
        # print(anglelist)
        # print(lena)
        # print(center)
            
        feature = [count_finger,count_defects,conarea/hullarea,anglelist[0],anglelist[1],anglelist[2],anglelist[3],lena[0],lena[1],lena[2],lena[3]]#,count_defects,anglelist,lena,center
        x.append(feature)
        y.append(fname)

      

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import linear_model
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.25, random_state=46)
C = 1.0
#**81.25%
knn = KNeighborsClassifier(n_neighbors=10)  
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print("Accuracyknn:",metrics.accuracy_score(y_test, y_pred))

svc = svm.SVC(kernel='linear', C=C).fit(x_train, y_train)
y_pred = svc.predict(x_test)
print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))

rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(x_train, y_train)
y_pred = rbf_svc.predict(x_test)
print("Accuracysvcrbf:",metrics.accuracy_score(y_test, y_pred))

poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(x_train, y_train)
y_pred = poly_svc.predict(x_test)
print("Accuracysvcpoly:",metrics.accuracy_score(y_test, y_pred))

#**75%
lin_svc = svm.LinearSVC(C=C).fit(x_train, y_train)
lin_svc.fit(x_train, y_train)
y_pred = lin_svc.predict(x_test)
print("Accuracysvclin:",metrics.accuracy_score(y_test, y_pred))

clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))



logreg.fit(X, Y)

Accuracyknn: 0.8125
Accuracysvc: 0.5833333333333334
Accuracysvcrbf: 0.5833333333333334
Accuracysvcpoly: 0.3958333333333333
Accuracysvclin: 0.75
Accuracysvc: 0.5625
